In [ ]:
"""
Using the same code found in.
https://github.com/fellowship/platform-demos2/blob/master/fashion-apparel/streetstyle/ss27k_preprocess.py

"""
import argparse
import pandas as pd
import numpy as np
from PIL import Image
import os
import torch
import fastai
from tqdm import tqdm
import matplotlib.patches as patches
#from google.colab import drive
torch.cuda.is_available()

In [ ]:
print(f'fastai version is {fastai.__version__}')
#drive.mount('/gdrive')

In [ ]:
#Setting the directory location
#The processed_FN will be the cropped manifest
STREETSTYLE_DIR = "/gdrive/My Drive/streetstyle27k_cropped"
MANIFEST_FN = "streetstyle27k.manifest"
IMAGES_PATH = "streetstyle27k"
PROCESSED_FN = "streetstyle27k.processed.manifest"
labels = pd.read_csv(os.path.join(STREETSTYLE_DIR, MANIFEST_FN))
print("Found {} entries in {}".format(len(labels), MANIFEST_FN))

In [ ]:
label_cols = ['clothing_pattern', 'major_color',
       'wearing_necktie', 'collar_presence', 'wearing_scarf', 'sleeve_length',
       'neckline_shape', 'clothing_category', 'wearing_jacket', 'wearing_hat',
       'wearing_glasses', 'multiple_layers']
# Replacing spaces with "_", converting all labels to lower case labels, NA to no label
# replacing the more_than_1_color label to multicolored
for col in labels.columns:
    if col in set(label_cols):
        labels[col] = (labels[col].str.lower()
                                  .str.replace(" ", "_")
                                  .str.replace("more_than_1_color", "multicolored")
                                  .fillna("no_label"))
        
print(labels.columns)

In [ ]:
images = []
do_cropping = True 
for _, item in labels[["url", "x1", "x2", "y1", "y2"]].iterrows():
    
    uri = item["url"]
    fn = os.path.basename(uri)
    # The path (url)of each image of the images in the manifest starts with the folders its stored e.g. 20f,20d.
    img_path = os.path.join(STREETSTYLE_DIR, IMAGES_PATH, fn[0], fn[1], fn[2], fn)
    cropped_img_path = img_path.replace("/" + IMAGES_PATH + "/", "/" + IMAGES_PATH + "_cropped/")
    if do_cropping and not os.path.exists(cropped_img_path):
        bbox = item["x1"], item["y1"], item["x2"], item["y2"]
        im = Image.open(img_path).crop(bbox)
        os.makedirs(os.path.dirname(cropped_img_path), exist_ok=True)
        im.save(cropped_img_path)
    images.append(cropped_img_path.replace("{}/".format(STREETSTYLE_DIR), ""))

In [ ]:
labels["images"] = images
fastai_labels = []

for _, item in labels[label_cols].iterrows():
    #print(item.tolist())
    item = ["{}_{}".format(c, v) for c, v in zip(label_cols, item.tolist())]
    fastai_labels.append(" ".join(item))
labels["fastai"] = fastai_labels

labels[["images", "fastai"]].to_csv(os.path.join(STREETSTYLE_DIR, PROCESSED_FN), index=False)
print("Writing csv file to {}".format(os.path.join(STREETSTYLE_DIR, PROCESSED_FN)))
labels.to_csv("streetstyle.preprocess.csv", index=False)



In [ ]:
import fastai
from fastai.imports import *
from fastai import *
from fastai.vision import *
from tqdm import tqdm
import pandas as pd
import numpy as np
from PIL import Image
import os
import torch

import matplotlib.patches as patches
STREETSTYLE_DIR = "/gdrive/My Drive/streetstyle27k_cropped"
MANIFEST_FN = "streetstyle27k.manifest"
IMAGES_PATH = "streetstyle27k"
PROCESSED_FN = "streetstyle27k.processed.manifest"
labels = pd.read_csv(os.path.join(STREETSTYLE_DIR, MANIFEST_FN))
print("Found {} entries in {}".format(len(labels), MANIFEST_FN))
categories = ['clothing_pattern', 'major_color', 'wearing_necktie', 'collar_presence', 
              'wearing_scarf', 'sleeve_length', 'neckline_shape', 'clothing_category', 
              'wearing_jacket', 'wearing_hat', 'wearing_glasses', 'multiple_layers']


np.random.seed(1254)
torch.manual_seed(1254)



basedir = Path(STREETSTYLE_DIR)
csvpath = 'streetstyle.preprocess.csv'
print(csvpath)

ds_tfms = get_transforms()
input_size = (336, 336)
df = pd.read_csv(csvpath)
print(df.columns)


In [ ]:
epochs_list = [3,10]
lrs_list =[1e-2, 5e-6]
do_lrfind = True
import torchvision.models as models

resnet18 = models.resnet18(pretrained=True)
#inception = models.inception_v3(pretrained=True)
#vgg16 = models.vgg16(pretrained=True)
modelname =  "baseline"
for c in categories[0:1]:
    print("Network for {}".format(c))
    df_category = df[["images", c]]
    df_category = df_category[~df_category[c].isna()]
    print(df_category.head())
    data = (ImageList.from_df(df_category, path=basedir)
                     .split_by_rand_pct(0.2)
                     .label_from_df()
                     .transform(ds_tfms, size=input_size)
                     .databunch(bs=16)
                     .normalize(imagenet_stats))
    #print(data)
    #learn = Learner(data, resnet18, metrics=[accuracy], callback_fns=[callbacks.CSVLogger])
    learn = cnn_learner(data, models.resnet34, metrics=[accuracy], callback_fns=[callbacks.CSVLogger])
    learn.freeze()
    for i, (epochs_i, lrs_i) in enumerate(zip(epochs_list, lrs_list)):
        if do_lrfind:
            print("Searching for optimal learning rates....")
            learn.lr_find()
            learn.recorder.plot()
            plt.savefig("lrfinder_{}_{}_{}.png".format(modelname, c, sum(epochs_list[:i+1])))
            if i != 0:
                learn.load("{}_{}_{}epochs".format(modelname, c, sum(epochs_list[:i]))) #fixed to i instead of i+1
        learn.fit_one_cycle(epochs_i, lrs_i)
        learn.save("{}_{}_{}epochs".format(modelname, c, sum(epochs_list[:i+1])))
        learn.unfreeze()

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(2, max_lr=slice(1e-7,1e-5))